In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
from numpy import linalg
from sklearn.linear_model import Lasso, Ridge

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [4]:
def STRidge(teta,ut,landa,tol,iters,normalize=2):

    n,d = teta.shape
    X = np.zeros((n,d))
    if normalize != 0:
        Mreg = np.zeros((d,1))
        for i in range(0,d):
            Mreg[i] = 1.0/(np.linalg.norm(teta[:,i], normalize))
            X[:,i] = Mreg[i]*teta[:,i]
    else: X = teta

    if landa!=0:
        w=linalg.lstsq(X.T@X+landa*np.eye(d),X.T@ut,rcond=None)[0]
    else:
        w=linalg.lstsq(X,ut,rcond=None)[0]
    num_relevant = d #initial guesses
    biginds = np.where( abs(w) > tol)[0]

    for j in range(iters):

        # Figure out which items to cut out
        smallinds = np.where( abs(w) < tol)[0]
        new_biginds = [i for i in range(d) if i not in smallinds]

        # If nothing changes then stop
        if num_relevant == len(new_biginds): break
        else: num_relevant = len(new_biginds)
        # Also make sure we didn't just lose all the coefficients
        if len(new_biginds) == 0:
            if j == 0:
                #if print_results: print "Tolerance too high - all coefficients set below tolerance"
                return w
            else: break
        biginds = new_biginds

        # Otherwise get a new guess
        w[smallinds] = 0
        if landa != 0: w[biginds] = np.linalg.lstsq(X[:, biginds].T@(X[:, biginds]) + landa*np.eye(len(biginds)),X[:, biginds].T@(ut),rcond=None)[0]
        else: w[biginds] = np.linalg.lstsq(X[:, biginds],ut,rcond=None)[0]

    # Now that we have the sparsity pattern, use standard least squares to get w
    if (list(biginds)!=[]):
        w[biginds] = np.linalg.lstsq(X[:, biginds],ut,rcond=None)[0]
    if normalize != 0:
        return np.multiply(np.ndarray.flatten(Mreg),w)
    else:
        return w

In [21]:
def TrainSTRidge(teta,ut,landa,d_tol,tol_iters=25,STR_iters=10,normalize=2):

    np.random.seed(0) # for consistency
    n,_ = teta.shape
    #train = np.random.choice(n, int(n*0.8), replace = False)
    #test = [i for i in np.arange(n) if i not in train]
    train=int(n*0.8)
    teta_train = teta[:train,:]
    teta_test = teta[train:,:]
    ut_train = ut[:train]
    ut_test = ut[train:]
    D = teta_train.shape[1]

    d_tol = float(d_tol)
    tol = d_tol
    l0_penalty=1e-3*linalg.cond(teta) #condition number


    # Get the standard least squares estimator
    w = np.zeros((D,1))
    w_best = np.linalg.lstsq(teta_train, ut_train,rcond=None)[0]
    err_best = np.linalg.norm(ut_test - teta_test@(w_best), 2) + l0_penalty * np.count_nonzero(w_best)
    tol_best = 0

    # Now increase tolerance until test performance decreases
    for iter in range(tol_iters):

        # Get a set of coefficients and error
        w = STRidge(teta_train, ut_train, landa, tol, STR_iters, normalize)
        err = np.linalg.norm(ut_test - teta_test@(w), 2) + l0_penalty*np.count_nonzero(w)

        # Has the accuracy improved?
        if err <= err_best:
            err_best = err
            w_best = w
            tol_best = tol
            tol = tol + d_tol

        else:
            tol = max([0,tol - 2*d_tol])
            d_tol  = 2*d_tol / (tol_iters - iter)
            tol = tol + d_tol
    print(tol_best)
    return w_best


In [6]:
def Finite_diff(u,dx, ord):
    """Requires:
    -the vector to calculate the derivative of  (1D np.array)
    -the finite step
    -the order of the derivative

    Returns:
    -the derivative as a vector of the same dimension
    """
    n=u.size
    d_u=np.zeros(n)

    if ord==1:
        for i in range(1,n-1):
           d_u[i] = (u[i+1]-u[i-1]) / (2*dx)  #central differences

        d_u[0] = (-3.0/2*u[0] + 2*u[1] - u[2]/2) / dx #second order method for forward finite differences
        d_u[n-1] = (3.0/2*u[n-1] - 2*u[n-2] + u[n-3]/2) / dx #second order method for backward finite differences
        return d_u

    if ord==2:
        for i in range(1,n-1):
            d_u[i] = (u[i+1]-2*u[i]+u[i-1]) / dx**2

        d_u[0] = (2*u[0] - 5*u[1] + 4*u[2] - u[3]) / dx**2
        d_u[n-1] = (2*u[n-1] - 5*u[n-2] + 4*u[n-3] - u[n-4]) / dx**2
        return d_u

    if ord==3:
        for i in range(2,n-2):
            d_u[i] = (u[i+2]/2-u[i+1]+u[i-1]-u[i-2]/2) / dx**3

        d_u[0] = (-2.5*u[0]+9*u[1]-12*u[2]+7*u[3]-1.5*u[4]) / dx**3
        d_u[1] = (-2.5*u[1]+9*u[2]-12*u[3]+7*u[4]-1.5*u[5]) / dx**3
        d_u[n-1] = (2.5*u[n-1]-9*u[n-2]+12*u[n-3]-7*u[n-4]+1.5*u[n-5]) / dx**3
        d_u[n-2] = (2.5*u[n-2]-9*u[n-3]+12*u[n-4]-7*u[n-5]+1.5*u[n-6]) / dx**3
        return d_u

    if ord>3:
        return Finite_diff(Finite_diff(u,dx,3), dx, ord-3)

In [7]:
def D_library_1d(u,ux,uxx,uxxx, desc=['u','ux','uxx','uxxx'],max_exp=2):
    """ Requires:
    -the input function and its derivatives (1d numpy arrays)
    -a list containing their names
    -the maximum exponent desired (more than 3 makes the number of vectors dramatically increase)
    -a list description of their name (change only if you change the order of the inputs)

    Returns:
    -the teta matrix
    -the list description of each column
    """
    n=u.size
    assert(n==ux.size)
    assert(n==uxx.size)
    assert(n==uxxx.size)
    f_list=list((u,ux,uxx,uxxx)) #wraps vectors in an iterable
    teta=np.expand_dims(np.ones(n),axis=1)   #initilize teta with the constant term
    teta_desc=['constant']
    dim=len(f_list)
    #adding the vectors and their powers
    for i in range(dim):
        for esp in range(1,max_exp+1):  #from 1 (the vectors) to max_exp
            c=np.expand_dims(f_list[i]**esp,axis=1)
            teta=np.concatenate((teta,c),axis=1)  #attaches it column wise
            if esp == 1:
                teta_desc.append(desc[i])
            else:
                teta_desc.append(desc[i]+"^"+str(esp))
    #cross products between powers of the vectors
    if max_exp==1:
        assert(teta[0,:].shape[0]==len(teta_desc))
        return teta,teta_desc
    for i in range(1,max_exp):                     #for every exponent starting from 1 and ending with max_exp-1
        for current in range(dim):                     #for every vector
            if current!=dim-1:                         #checks if it's not last
                for j in range(current+1,dim):         #for all the next vectors
                    if i==1:
                        a=np.expand_dims(f_list[current]*f_list[j], axis=1)  #simply multyplying (needs to expand otherwise concatenate doesn't work)
                        teta=np.concatenate((teta,a),axis=1)
                        teta_desc.append(desc[current]+"_"+desc[j])
                    else:
                        d=np.expand_dims((f_list[current]**i)*f_list[j], axis=1)  #elevates one and multiplies by the other
                        teta=np.concatenate((teta,d),axis=1)
                        teta_desc.append(desc[current]+"^"+str(i)+"_"+desc[j])

                        b=np.expand_dims((f_list[current]*(f_list[j]**i)),axis=1) #does the inverse since i!=1
                        teta=np.concatenate((teta,b),axis=1)
                        teta_desc.append(desc[current]+"_"+desc[j]+"^"+str(i))
    assert(teta[0,:].shape[0]==len(teta_desc))

    return teta,teta_desc


In [8]:
def Computing_mixed_derivatives(u,dx,dy,tag='u'):
    """Inputs:
    -u needs to be a 2+1D input (first 2 spatial, last temporal)
    -the finite steps
    Returns:
    -a list of all (flattened) non redundant mixed derivatives of u
    """
    dim=u.shape
    assert(dim[0]==dim[1]) #the grid needs to be evenly spaced on a square
    ux=np.zeros(dim)
    uxx=np.zeros(dim)
    uxxx=np.zeros(dim)
    uy=np.zeros(dim)
    uyy=np.zeros(dim)
    uyyy=np.zeros(dim)
    uxy=np.zeros(dim)
    uxyy=np.zeros(dim)
    uyxx=np.zeros(dim)
    #normal derivatives
    for var in range(dim[0]):
      for t in range(dim[2]):
        ux[:,var,t]=Finite_diff(u[:,var,t],dx,1)  #du/dx
        uxx[:,var,t]=Finite_diff(u[:,var,t],dx,2)
        uxxx[:,var,t]=Finite_diff(u[:,var,t],dx,3)
        uy[var,:,t]=Finite_diff(u[var,:,t],dy,1)  #du/dy
        uyy[var,:,t]=Finite_diff(u[var,:,t],dy,2)
        uyyy[var,:,t]=Finite_diff(u[var,:,t],dy,3)
    #mixed derivatives
    for x in range(dim[0]):
        for t in range(dim[2]):
            uxy[x,:,t]=Finite_diff(ux[x,:,t],dy,1)
            uxyy[x,:,t]=Finite_diff(ux[x,:,t],dy,2)
            uyxx[:,x,t]=Finite_diff(uy[:,x,t],dx,2)
    #Flattening the data
    u=np.ndarray.flatten(u)
    ux=np.ndarray.flatten(ux)
    uxx=np.ndarray.flatten(uxx)
    uxxx=np.ndarray.flatten(uxxx)
    uy=np.ndarray.flatten(uy)
    uyy=np.ndarray.flatten(uyy)
    uyyy=np.ndarray.flatten(uyyy)
    uxy=np.ndarray.flatten(uxy)
    uxyy=np.ndarray.flatten(uxyy)
    uyxx=np.ndarray.flatten(uyxx)

    return [u,ux,uxx,uxxx,uy,uyy,uyyy,uxy,uxyy,uyxx],[tag,tag+'x',tag+'xx',tag+'xxx',tag+'y',tag+'yy',tag+'yyy',tag+'xy',tag+'xyy',tag+'yxx']


In [9]:
def D_library_2d(u,v,dx,dy,max_exp=2):
    """Inputs:
    -the two components of a vector field 2+1D->2D (data need to be three dimensional)
    -the two finite steps for the finite differences
    -the max order of the desired polynomial (more than 2 causes the number to dramatically increase)
    Returns:
    -the teta matrix and its description:
    """
    dim1=u.shape
    assert(dim1==v.shape)
    #computing all mixed derivatives
    u_der, u_desc=Computing_mixed_derivatives(u,dx,dy)
    v_der, v_desc=Computing_mixed_derivatives(v,dx,dy,'v')
    teta=np.expand_dims(np.ones(u.size),axis=1)
    teta_desc=['constant']
    n=len(u_der)
    #adding singular powers of the vectors u and v
    for i in range(n):
        for esp in range(1,max_exp+1):
            c=np.expand_dims(u_der[i]**esp,axis=1)
            teta=np.concatenate((teta,c),axis=1)  #attaches it column wise
            e=np.expand_dims(v_der[i]**esp,axis=1)
            teta=np.concatenate((teta,e),axis=1)
            if esp == 1:
                teta_desc.append(u_desc[i])
                teta_desc.append(v_desc[i])
            else:
                teta_desc.append(u_desc[i]+"^"+str(esp))
                teta_desc.append(v_desc[i]+'^'+str(esp))
    if max_exp==1:
        return teta,teta_desc
    #uniting the lists to loop over them
    desc=[]
    all_der=[]
    for i in range(n):
        all_der.append(u_der[i])
        all_der.append(v_der[i])
        desc.append(u_desc[i])
        desc.append(v_desc[i])

    for i in range(1,max_exp):                     #for every exponent starting from 1 and ending with max_exp-1
        for current in range(2*n):                     #for every vector(there are 2n vectors)
            if current!=2*n-1:                         #checks if it's not last
                for j in range(current+1,2*n):         #for all the next vectors
                    if i==1:
                        a=np.expand_dims(all_der[current]*all_der[j], axis=1)  #simply multyplying
                        teta=np.concatenate((teta,a),axis=1)
                        teta_desc.append(desc[current]+"_"+desc[j])
                    else:
                        d=np.expand_dims((all_der[current]**i)*all_der[j], axis=1)  #elevates one and multiplies by the other
                        teta=np.concatenate((teta,d),axis=1)
                        teta_desc.append(desc[current]+"^"+str(i)+"_"+desc[j])

                        b=np.expand_dims((all_der[current]*(all_der[j]**i)),axis=1) #does the inverse since i!=1
                        teta=np.concatenate((teta,b),axis=1)
                        teta_desc.append(desc[current]+"_"+desc[j]+"^"+str(i))
    return teta,teta_desc


In [10]:
def Pde_find(u,dx,dt,dimensions=1,lam=10**-4,d_tol1=1,d_tol2=None,tol_iter=25,STR_iters=10,v=None,dy=None,normalize=2,max_exp=2):
    """ Requires:
    -the data as a 1+1d or 2+1d data to be fit as a PDE (spatial first then time)
    -the temporal and the spatial steps (dy is optional if is one d)
    -the l0 penalty
    -the initial tolerance
    -other parameters for the STRidge method

    Returns:
    -the sparse solution of ut=teta*csi
    -the description of the relevant found parameters
    if the data is 2_d, both the parameters and desc are returned
    """
    #calculate spatial derivatives at every time step
    if dimensions==1:
        dims=u.shape
        n=dims[0] #the dim of the spatial grid
        m=dims[1] #dim of temporal grid
        ut=np.zeros(dims)   #instantiates space for the derivatives
        ux=np.zeros(dims)
        uxx=np.zeros(dims)
        uxxx=np.zeros(dims)
        for t in range(m):
            ux[:,t]=Finite_diff(u[:,t],dx,ord=1) #compute spatial der at each time step
            uxx[:,t]=Finite_diff(u[:,t],dx,ord=2)
            uxxx[:,t]=Finite_diff(u[:,t],dx,ord=3)

        #compute ut for every x in the grid
        for x in range(n):
            ut[x,:]=Finite_diff(u[x,:],dt,1)

        #Flatten all vectors to build the system
        u=np.ndarray.flatten(u)
        ut=np.ndarray.flatten(ut)
        ux=np.ndarray.flatten(ux)
        uxx=np.ndarray.flatten(uxx)
        uxxx=np.ndarray.flatten(uxxx)

        #build teta
        teta, teta_desc=D_library_1d(u,ux,uxx,uxxx, max_exp=max_exp)
        print(len(teta[0,:]))
        #Find the sparse solution to the system Ut=teta*csi
        csi=TrainSTRidge(teta, ut, lam, d_tol1, tol_iter, STR_iters, normalize)

        rel_ind=np.nonzero(csi)[0]
        rel_desc=[]
        for p in rel_ind:
          rel_desc.append(teta_desc[p])
        rel_coeff=csi[rel_ind]

        return rel_coeff, rel_desc

    elif dimensions==2:
        # computing ut and vt
        dims=u.shape
        assert(dims==v.shape)
        ut=np.zeros(dims)
        vt=np.zeros(dims)
        for x in range(dims[0]):
            for y in range(dims[1]):
                ut[x,y,:]=Finite_diff(u[x,y,:],dt,1)
                vt[x,y,:]=Finite_diff(v[x,y,:],dt,1)
        #flattening ut and vt to costruct systems
        ut=np.ndarray.flatten(ut)
        vt=np.ndarray.flatten(vt)
        #constructing teta and the description of its columns
        if dy==None:
            dy=dx
        teta, teta_desc= D_library_2d(u,v,dx,dy,max_exp)
        print(len(teta[0,:]))
        if d_tol2==None:
            d_tol2=d_tol1

        csi1=TrainSTRidge(teta, ut, lam, d_tol1, tol_iter, STR_iters, normalize)
        csi2=TrainSTRidge(teta, vt, lam, d_tol2, tol_iter, STR_iters, normalize)

        #checking the solution
        rel_ind1=np.nonzero(csi1)[0]
        rel_ind2=np.nonzero(csi2)[0]
        rel_desc1=[]
        rel_desc2=[]
        for p in rel_ind1:
            rel_desc1.append(teta_desc[p])
        for p in rel_ind2:
            rel_desc2.append(teta_desc[p])

        return csi1[rel_ind1],rel_desc1, csi2[rel_ind2],rel_desc2
    else:
        raise ValueError("Only up to 2 spatial dimensions!")





First Dataset

In [11]:
data1=np.load('1.npz')
u=data1['u']
t_grid=data1['t'][0,:]
x_grid=data1['x'][:,0]
dx1=x_grid[1]-x_grid[0]
dt1=t_grid[1]-t_grid[0]

In [71]:
coeff, desc= Pde_find(u,dx1,dt1,lam=10**-4,d_tol1=0.02, tol_iter=50, max_exp=2)
print(coeff,desc)

15
1.0000000000000004
[ 0.1002203  -1.00098715] ['uxx', 'u_ux']


It's Burger's equation with mu=1/10!

Second dataset:

In [14]:
data2=np.load('2.npz')
u2=data2['u']
t_grid2=data2['t'][0,:]
x_grid2=data2['x'][:,0]
dx2=x_grid2[1]-x_grid2[0]
dt2=t_grid2[1]-x_grid2[0]
print(u2.shape)

(512, 201)


In [79]:
coeff2,desc2=Pde_find(u2, dx2, dt2, lam=10**-4, d_tol1=0.001,tol_iter=25, STR_iters=25, max_exp=2)
print(coeff2)
print(desc2)

15
0.025000000000000015
[-0.00330799 -0.01986445]
['uxxx', 'u_ux']


In [45]:
coeff2,desc2=Pde_find(u2, dx2, dt2, lam=10**-5, d_tol1=5, max_exp=3)
print(coeff2)
print(desc2)

31
0.06666666666666667
[0.01343242]
['u^2_uxxx']


In [16]:
data3=np.load('3.npz')
u3=data3['u']
v3=data3['v']
x_grid3=data3['x'][:,0,0]
y_grid3=data3['y'][0,:,0]
t_grid3=data3['t'][0,0,:]
dx=x_grid3[1]-x_grid3[0]
dy=y_grid3[1]-y_grid3[0]
dt=t_grid3[1]-t_grid3[0]

In [81]:
coeff_u,desc_u,coeff_v,desc_v=Pde_find(u3,dx,dt,dimensions=2,lam=10**-5,d_tol1=1,d_tol2=5,v=v3,dy=dy,max_exp=1)
print(coeff_u)
print(desc_u)
print(coeff_v)
print(desc_v)

21
20.599999999999994
88.75
[ 0.05451305  0.8937079   0.05084615  0.01114982 -0.00309983  0.05478725]
['u', 'v', 'uxx', 'vxx', 'uxxx', 'uyy']
[-0.8879761   0.0610512   0.05344465  0.06628146]
['u', 'v', 'vxx', 'vyy']
